In [1]:
print("hare krishna")

hare krishna


### Imports

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq

#### API Keys

In [2]:
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
groq_api_key=os.getenv('GROQ_API_KEY')

#### Embeedings

In [3]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="lala",
    openai_api_version="2024-03-01-preview",
)

### Create Vector Data base and store

In [28]:
def add_vector_database(directory_path):
    try:
        # Load documents
        loader = PyPDFDirectoryLoader(directory_path)
        docs = loader.load()
        if not docs:
            raise ValueError("No documents found in the directory.")

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
        documents = text_splitter.split_documents(docs)
        if not documents:
            raise ValueError("Document splitting resulted in an empty list.")

        # Generate embeddings
        texts = [doc.page_content for doc in documents]
        embeddings_list = embeddings.embed_documents(texts)
        if not embeddings_list:
            raise ValueError("Embeddings generation resulted in an empty list.")

        # Create FAISS database
        db = FAISS.from_documents(documents, embeddings)
        database = FAISS.load_local('first__vector', embeddings, allow_dangerous_deserialization=True)
        database.merge_from(db)
        database.save_local('first__vector')

    except Exception as e:
        print(f"An error occurred: {e}")

#### llm,prompt,chain


In [7]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

document_chain=create_stuff_documents_chain(llm,prompt)


#### database load and answer

In [15]:
def answer(input):
    llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

    prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context. 
    Think step by step before providing a detailed answer. 
    I will tip you $1000 if the user finds the answer helpful. 
    <context>
    {context}
    </context>
    Question: {input}""")

    document_chain=create_stuff_documents_chain(llm,prompt)
    database = FAISS.load_local('first__vector',embeddings, allow_dangerous_deserialization= True)
    retriever=database.as_retriever()
    retrieval_chain=create_retrieval_chain(retriever,document_chain)
    response=retrieval_chain.invoke({"input":input})

    return response['answer']

In [58]:
answer("In 2022, the ACS national poverty rate was ?")

'According to the provided context, the answer to the question is:\n\nIn 2022, the ACS national poverty rate was 12.6 percent.'

In [54]:
# add_vector_database("attention_dir")

In [56]:
add_vector_database("us_census")

In [71]:
answer("The percentage of the U.S. population with income below 50 percent of their poverty threshold was ?")

"I'm happy to help! However, I don't see any context provided. Please provide the context, and I'll do my best to answer the question step by step."

In [64]:
answer("Tell me about Changes in the Uninsured Rate by 25 Most Populous Metropolitan Areas from 2021 to 2022 ?")

'Based on the provided context, here is the answer to your question:\n\nAccording to the text, between 2021 and 2022, 11 out of the 25 most populous metropolitan areas in the United States saw declines in the uninsured rate, and none experienced an increase in uninsured rates. Specifically, Charlotte-Concord-Gastonia, NC-SC, experienced one of the largest decreases (1.8 percentage points) in the uninsured rate from 2021 to 2022.\n\nAdditionally, the text mentions that California expanded Medi-Cal to all adults 50 years or older in May 2022, which may contribute to the higher insured rates in this metropolitan area.'

In [51]:
answer("Tell me about near poverty")

'Based on the provided context, here is the answer to the question:\n\nNear poverty refers to individuals with an income-to-poverty ratio between 100 percent to below 125 percent of their poverty threshold. This means that people who are classified as being in near poverty have an income that is above their poverty threshold, but still relatively close to it.'

In [59]:
answer("Tell me about sections of cyber laws")

'Based on the provided context, I will answer the question step by step.\n\nThe context mentions a few cases related to cyber laws, specifically sections of the Information Technology (IT) Act and the Cable TV Networks Regulation Act, 1995. \n\nSection 67 of the Information Technology (IT) Act is mentioned as a section under which charges were sought to be quashed in the Rajendra Agrawal v. State of Chhattisgarh case.\n\nSection 500 of the IPC is also mentioned as a section under which charges were sought to be quashed in the same case.\n\nSection 482 of the Cr PC is mentioned as the section under which the quashing of charges was sought in the Rajendra Agrawal v. State of Chhattisgarh case.\n\nThere is no specific information about other sections of the cyber laws in the context provided. \n\nHowever, the context does mention some other cases related to cyber laws, such as Praveen Arimbrathodiyil v. Union of India and Union of India v. Sudesh Kumar Singh, which may involve other secti

In [60]:
answer("Tell me about section 65B")

'Based on the provided context, I can answer the question about Section 65B of the Evidence Act.\n\nSection 65B of the Evidence Act deals with the admissibility and contents of electronic evidence. Specifically, it states that a section 65B(4) certificate is mandatory for secondary evidence and can be given by a person in a responsible position related to device operation or management.\n\nFurthermore, the context mentions that Section 65B(4) is a condition precedent to the admissibility of evidence by way of electronic record, and that the court relied on previous rulings to hold that the prosecution should be relieved of the obligation to provide a section 65B(4) certificate if they have made efforts to obtain it but have no control over the relevant third-party companies.\n\nOverall, Section 65B of the Evidence Act is crucial in determining the admissibility of electronic evidence in court, and it requires a certificate from a person in a responsible position related to device opera

In [55]:
answer("what in encoder and decoder")

'Based on the provided context, the answer to the question is:\n\nIn the encoder and decoder, both use stacked self-attention and point-wise, fully connected layers.\n\n* The encoder is composed of a stack of N=6 identical layers, each with two sub-layers:\n\t+ The first sub-layer is a multi-head self-attention mechanism.\n\t+ The second sub-layer is a simple, position-wise fully connected feed-forward network.\n* The decoder also follows a similar architecture, with a stack of identical layers, each with two sub-layers:\n\t+ The first sub-layer is a multi-head self-attention mechanism, using the queries from the previous decoder layer.\n\t+ The second sub-layer is a simple, position-wise fully connected feed-forward network.\n\nI hope this answer helps you!'

In [12]:
# add_vector_database('attention_dir')

In [61]:
answer("Transformer follows this overall architecture using which stacked ?")

'Based on the provided context, the answer is:\n\n**Encoder-Decoder structure**\n\nThe text states: "Most competitive neural sequence transduction models have an encoder-decoder structure [5,2,35]." This suggests that the Transformer model follows a similar architecture, which consists of an encoder that maps an input sequence to a sequence of continuous representations, and a decoder that generates an output sequence one element at a time.'

#### Delete

In [4]:
data = FAISS.load_local('first__vector',embeddings, allow_dangerous_deserialization= True)
query="Attention is all you need"
result=data.similarity_search(query)
result[0].page_content

'Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3'

In [49]:
data.docstore._dict

{'72e0d57c-76ca-4edc-9ec4-68def665f041': Document(page_content='Input-Input Layer5\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the\nsentence. We give two such examples above, from two different heads from the encoder self-attention\nat layer 5 of 6. The heads clearly learned to perform different tasks.\n15', meta

In [66]:
db = FAISS.load_local('first__vector',embeddings, allow_dangerous_deserialization= True)
print("count before:", db.index.ntotal)
# db.delete([db.index_to_docstore_id[0]])
# print("count after:", db.index.ntotal)

count before: 358


In [35]:
print("count before:", db.index.ntotal)
db.delete([db.index_to_docstore_id[427]])
print("count after:", db.index.ntotal)

count before: 428
count after: 427


In [67]:
print("count after:", db.index.ntotal)

count after: 358


In [68]:
for i in range(db.index.ntotal - 1, -1, -1):
    try:
        # Get the docstore ID for the current index
        doc_id = db.index_to_docstore_id[i]
        
        # Delete the entry from FAISS index and docstore
        db.delete([doc_id])
        
        # Add the index to the deleted list
        # deleted_indices.append(i)
    except Exception as e:
        print(f"Error deleting index {i}: {e}")


In [69]:
print("count after:", db.index.ntotal)

count after: 0


In [70]:
db.save_local('first__vector')